In [1]:
using JLD2
using FourierFlows
using CairoMakie
using AbstractFFTs
using Printf
using Interpolations
using LsqFit

include("../analysis/Notebooks/AnalysisUtils.jl")

mapfilter_snapshots (generic function with 1 method)

In [25]:
function load_slice(ω_size, idx)
    grid = TwoDGrid(; nx=512, Lx=2π)
    Ch_grid = zeros(ω_size, grid.nkr, grid.nl)
    file = jldopen(@sprintf("%s/%d/radial_data_k=%03d.jld2", fourier_directory, 1, 1))
    full_ω_size = size(file["ψt"], 1)
    println(full_ω_size)

    ω_idxs = (1:ω_size) .+ ((idx-1) * ω_size) .+ 2 # Skip for factoring reasons
    for file_idx=1:17
        file = jldopen(@sprintf("%s/%d/radial_data_k=%03d.jld2", fourier_directory, floor(Int, (file_idx-1)/4 + 1), file_idx))
        ψt = file["ψt"]
        @views Ch_grid[:, file_idx, :] .=  abs2.(ψt[ω_idxs, :])
    end
    ρsChrs = FourierFlows.radialspectrum.(eachslice(Ch_grid, dims=1), Ref(grid))
    return ρsChrs
end



load_slice (generic function with 1 method)

In [31]:
run_directory = "/vast/nad9961/swqg/2903911/"
fourier_directory = "/scratch/nad9961/swqg_fourier/2910505/"

Chr = zeros(1024, 30180)
for idx=1:30
    @time result = load_slice(1006, idx);
    for slice_idx = 1:1006
        @views Chr[:, 1006 * (idx - 1) + slice_idx] .= result[slice_idx][2][:]
    end
    println("Index ", idx, " done")
end

30182
 33.919736 seconds (31.35 k allocations: 10.845 GiB, 3.51% gc time)
Index 1 done
30182
 33.749933 seconds (31.35 k allocations: 10.845 GiB, 3.43% gc time)
Index 2 done
30182
 33.523126 seconds (31.35 k allocations: 10.845 GiB, 3.49% gc time)
Index 3 done
30182
 33.488994 seconds (31.35 k allocations: 10.845 GiB, 3.52% gc time)
Index 4 done
30182
 33.645812 seconds (31.35 k allocations: 10.845 GiB, 3.61% gc time)
Index 5 done
30182
 33.632550 seconds (31.35 k allocations: 10.845 GiB, 3.60% gc time)
Index 6 done
30182
 33.673522 seconds (31.35 k allocations: 10.845 GiB, 3.66% gc time)
Index 7 done
30182
 33.606353 seconds (31.35 k allocations: 10.845 GiB, 3.58% gc time)
Index 8 done
30182
 33.597930 seconds (31.35 k allocations: 10.845 GiB, 3.51% gc time)
Index 9 done
30182
 33.513219 seconds (31.35 k allocations: 10.845 GiB, 3.49% gc time)
Index 10 done
30182
 33.717085 seconds (31.35 k allocations: 10.845 GiB, 3.72% gc time)
Index 11 done
30182
 33.528146 seconds (31.35 k allocat

In [ ]:
ωs = 0.003158115705240993:0.003158115705240993:47.65596599208659
Ks = result[1][1][2:end]
f = Figure()
ax = Axis(f[1,1]; xscale=log10, yscale=log10)
hm = heatmap!(ax, Ks, ωs, log.(replace(Chr[2:end,2:ω_pos+1], 0.0=>NaN)))

f

In [75]:
length(ωs)

15090

In [83]:
size(Chr[:,2:ω_pos+1])

(1024, 15090)

In [82]:
length( 0.003158115705240993:0.003158115705240993:47.65596599208659)

15090

In [84]:
size(Ks)

(1023,)